# LSTM - Generación de Texto

Importamos TensorFlow y otras librerías

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# !pip uninstall tensorflow tensorflow-gpu 
# !pip install tensorflow tensorflow-gpu
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
import re

###Leer los datos
Cargamos el conjunto de datos por cada script. Y limpiamos los caracteres que no nos interesan.

In [0]:
text = ''
codes = ['ps1.py', 'ps2.py', 'ps3.py']

for code in codes:
  text = text + open(code, 'rb').read().decode(encoding='latin-1')
  
text = text.lower()
text = re.sub('#.*\n', '', text)
text = re.sub('""".*?"""', '', text, flags=re.DOTALL)
text = re.sub(r'(\n\s*)+\n+', '\n\n', text, flags=re.DOTALL)
text = re.sub(r'\r', '', text, flags=re.DOTALL)
text = text[1:]
text = text.replace('"', "'")

print('Length of text: {} characters'.format(len(text)))
print(text[:500])

Length of text: 9035 characters
import math
import random

vowels = 'aeiou'
consonants = 'bcdfghjklmnpqrstvwxyz'
hand_size = 7

scrabble_letter_values = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 
    'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1,
    's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10
}

wordlist_filename = 'words.txt'

def load_words():

    print('loading word list from file...')
        infile = open(wordlist_filename, 'r')
        wo


Obtenemos los caracteres únicos de los scripts.

In [0]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
print(vocab)

59 unique characters
['\n', ' ', '!', '%', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', ':', '<', '=', '>', '?', '[', '\\', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}']


### Vectorizar el texto
Antes de entrenar, necesitamos asignar nuestras cadenas a una representación numérica. Creamos dos tablas de búsqueda: una que asigne caracteres a números y otra para números a caracteres.

In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

Ahora tenemos una representación entera para cada caracter. 

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '%' :   3,
  "'" :   4,
  '(' :   5,
  ')' :   6,
  '*' :   7,
  '+' :   8,
  ',' :   9,
  '-' :  10,
  '.' :  11,
  '/' :  12,
  '0' :  13,
  '1' :  14,
  '2' :  15,
  '3' :  16,
  '4' :  17,
  '5' :  18,
  '6' :  19,
  ...
}


In [0]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'import math\ni' ---- characters mapped to int ---- > [39 43 46 45 48 50  1 43 31 50 38  0 39]


### Crear ejemplos de entrenamiento y targets.
A continuación dividir el texto en secuencias de ejemplo. Cada secuencia de entrada contendrá seq_length caracteres del texto.

Para cada secuencia de entrada, los objetivos correspondientes contienen la misma longitud de texto, excepto el desplazamiento de un carácter a la derecha.

Entonces rompemos el texto en trozos de seq_length + 1. Por ejemplo, digamos que seq_length es 3 y nuestro texto es "Hola". La secuencia de entrada sería "Hol" y la secuencia de destino "ola".

In [0]:
seq_length = 100
examples_per_epoch = len(text)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(7):
  print(idx2char[i.numpy()])

i
m
p
o
r
t
 


El método ***batch*** nos permite convertir fácilmente estos caracteres individuales en secuencias del tamaño deseado.

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
print(len(list(sequences)))
      
for item in sequences.take(3):
  print(repr(''.join(idx2char[item.numpy()])))

89
"import math\nimport random\n\nvowels = 'aeiou'\nconsonants = 'bcdfghjklmnpqrstvwxyz'\nhand_size = 7\n\nscrab"
"ble_letter_values = {\n    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, \n  "
"  'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1,\n    's': 1, 't': 1, 'u': 1"


Para cada secuencia,  lo duplicamos y movemos para formar el texto de entrada y destino utilizando el método ***map*** para aplicar una función simple a cada lote.

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "import math\nimport random\n\nvowels = 'aeiou'\nconsonants = 'bcdfghjklmnpqrstvwxyz'\nhand_size = 7\n\nscra"
Target data: "mport math\nimport random\n\nvowels = 'aeiou'\nconsonants = 'bcdfghjklmnpqrstvwxyz'\nhand_size = 7\n\nscrab"


Cada índice de estos vectores se procesa como un paso de tiempo. Para la entrada en el paso de tiempo 0, el modelo recibe el índice para "i" y trata de predecir el índice para "m" como el siguiente carácter. En el siguiente paso de tiempo, hace lo mismo pero el RNN considera el contexto del paso anterior además del carácter de entrada actual.

In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[0:5], target_example[0:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 39 ('i')
  expected output: 43 ('m')
Step    1
  input: 43 ('m')
  expected output: 46 ('p')
Step    2
  input: 46 ('p')
  expected output: 45 ('o')
Step    3
  input: 45 ('o')
  expected output: 48 ('r')
Step    4
  input: 48 ('r')
  expected output: 50 ('t')


### Crear lotes de entrenamiento
Utilizamos ***tf.data*** para dividir el texto en secuencias manejables. Pero antes de introducir estos datos en el modelo, debemos barajar los datos y empaquetarlos en lotes.


In [0]:
BATCH_SIZE = 16
steps_per_epoch = examples_per_epoch

BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<DatasetV1Adapter shapes: ((16, 100), (16, 100)), types: (tf.int64, tf.int64)>

### Construir el modelo
Utilizamos tf.keras.Sequential para definir el modelo. Para este ejemplo simple se utilizan tres capas para definir nuestro modelo:

- **tf.keras.layers.Embedding:** La capa de entrada. Una tabla de búsqueda entrenable que asignará los números de cada carácter a un vector con dimensiones - embedding_dim.
- **tf.keras.layers.CuDNNLSTM:** Un tipo de RNN con unidades de tamaño = rnn_units.
- **Tf.keras.layers.Dense:** La capa de salida, con salidas vocab_size.

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

A continuación definimos una función para construir el modelo.

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        vocab_size, 
        embedding_dim, 
        batch_input_shape=[batch_size, None]),
    tf.keras.layers.CuDNNLSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size=len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

Para cada carácter, el modelo busca la incrustación (embedding), ejecuta LSTM en un paso de tiempo con la incrustación como entrada y aplica la capa densa para predecir la probabilidad del siguiente carácter.

### Prueba el modelo
Ahora ejecuta el modelo para ver que se comporta como se espera.

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(16, 100, 59) # (batch_size, sequence_length, vocab_size)


En el ejemplo anterior, la longitud de la secuencia de la entrada es 100, pero el modelo se puede ejecutar en entradas de cualquier longitud.

In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (16, None, 256)           15104     
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (16, None, 1024)          5251072   
_________________________________________________________________
dense_3 (Dense)              (16, None, 59)            60475     
Total params: 5,326,651
Trainable params: 5,326,651
Non-trainable params: 0
_________________________________________________________________


Para obtener predicciones reales del modelo necesitamos muestrear la distribución de salida, para obtener índices de caracteres reales. 
Es importante tomar muestras de esta distribución, ya que tomar el argmax de la distribución puede hacer que el modelo quede atrapado en un bucle.

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([15, 58, 16,  0,  3, 42,  2, 57, 51, 11, 40, 21,  5, 54, 31, 22, 39,
       40, 50, 34, 10,  5, 25, 46, 35, 28, 13, 50, 24,  3, 42, 24,  0, 54,
        3, 33, 26, 13, 52, 57, 25, 11, 15, 13, 19,  1, 39,  6, 56, 29,  2,
       20, 30, 26, 29, 27, 34, 42, 43, 42, 41, 26,  8, 52, 10, 30, 37, 42,
       23, 33, 42, 58, 41, 48, 31, 48, 35, 16, 26,  6, 41, 36, 48, 13, 15,
        8, 29, 15, 58, 10, 46, 57, 18, 32,  5,  9, 14, 49, 58,  7])

Decodificamos para ver el texto predicho por este modelo sin entrenamiento.

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))

Input: 
 ' 0) + 1\n\n    for i in range(num_vowels+1, n):    \n        x = random.choice(consonants)\n        hand'
Next Char Predictions: 
 '2}3\n%l!{u.j8(xa:ijtd-(>pe\\0t=%l=\nx%c?0v{>.206 i)z]!7_?][dlmlk?+v-_gl<cl}krare3?)kfr02+]2}-p{5b(,1s}*'


### Entrenar a la modelo
En este punto, el problema se puede tratar como un problema de clasificación. Dado el estado RNN anterior y la entrada en este paso de tiempo, predice la clase del siguiente carácter.

### Definir un optimizador, y una función de pérdida
La función de pérdida estándar tf.keras.losses.sparse_categorical_crossentropy funciona en este caso porque se aplica en la última dimensión de las predicciones.

Debido a que nuestro modelo devuelve logits (el vector de predicciones sin procesar), debemos establecer el distintivo from_logits.

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (16, 100, 59)  # (batch_size, sequence_length, vocab_size)
loss:       4.0763707


Configuramos el procedimiento de capacitación utilizando el método ***tf.keras.Model.compile***. Usaremos tf.train.AdamOptimizer con los argumentos predeterminados y la función de pérdida.

In [0]:
model.compile(optimizer = tf.train.AdamOptimizer(),loss = loss)

### Configurar puntos de control
Usamos ***tf.keras.callbacks.ModelCheckpoint*** para asegurarnos de que los puntos de control se guardan durante el entrenamiento.

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Ejecutar el entrenamiento
Usamos 5 épocas para entrenar el modelo. En Colab, establecemos el tiempo de ejecución en GPU para un entrenamiento más rápido.

In [0]:
EPOCHS = 3
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/5
9035/9035 [==============================] - 340s 38ms/step - loss: 0.0889
Epoch 2/5
9035/9035 [==============================] - 341s 38ms/step - loss: 0.0238
Epoch 3/5
9035/9035 [==============================] - 341s 38ms/step - loss: 0.0227
Epoch 4/5
 270/9035 [..............................] - ETA: 5:32 - loss: 0.0207

KeyboardInterrupt: ignored

### Generar texto
Restauramos el último punto de control.

Para mantener este paso de predicción simple, usamos un tamaño de lote de 1.

Debido a la forma en que el estado RNN se pasa de un paso de tiempo a otro, el modelo solo acepta un tamaño de lote fijo una vez construido.

Para ejecutar el modelo con un tamaño de lote diferente, necesitamos reconstruir el modelo y restaurar los pesos desde el punto de control.

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            15104     
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (1, None, 1024)           5251072   
_________________________________________________________________
dense_4 (Dense)              (1, None, 59)             60475     
Total params: 5,326,651
Trainable params: 5,326,651
Non-trainable params: 0
_________________________________________________________________


### El bucle de predicción
El siguiente bloque de código genera el texto:

- Comienza seleccionando una cadena de inicio, inicializando el estado RNN y configurando la cantidad de caracteres que se generarán.

- Obtenga la distribución de predicción del siguiente carácter utilizando la cadena de inicio y el estado RNN.

- Luego, use una distribución categórica para calcular el índice del carácter predicho. Utilice este carácter predicho como nuestra próxima entrada al modelo.

- El estado RNN devuelto por el modelo se retroalimenta en el modelo para que ahora tenga más contexto, en lugar de una sola palabra. Después de predecir la siguiente palabra, los estados RNN modificados se incorporan nuevamente al modelo, que es la forma en que aprende a medida que obtiene más contexto de las palabras predichas previamente.

- Para generar texto, la salida del modelo se retroalimenta a la entrada.

In [0]:
def generate_text(model, start_string):
  num_generate = 1030

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

print(generate_text(model, start_string="for "))

for x in sequence:
        freq[x] = freq.get(x,0) + 1
    for i in range(1, 37, 1):
        component1 += scrabble_letter_values.get(i,0)

    component2 = max(7*word_length-3*(n-word_le_surrent_savings += (current_savings*r/12 
                            + portion_saved*(local_annual_sand[i]

    return sum_

def play_hand(hand, word_list):

    new_hand = hand.copy()
    total_pointsavings(high) < portion_down_payment:
        print('it is not possible to pay the down payment in the cost of your dream home: '))
semi_annual_raise = float(input('insert the semi-annual raise, as decer = input('would you like to replay the hand? ')
            if answer in {'no', 'no', 'no'}:
           for x in word_list)

def calculate_handlen(hand):

    sum_ = 0
    for i in hand:
        sum_ += he cost of your dream home: '))
semi_annual_raise = float(input('insert the semi-annual raise, as decer = input('would you like to replay the hand? ')
            if answer in {'no', 'no', 'no'}:
         

### Referencias
- https://www.tensorflow.org/tutorials/sequences/text_generation